# Workshop in Classic Predictive Analytics

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from sklearn import cross_validation, feature_selection,preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, auc
from patsy import dmatrices

In [54]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.externals.six import StringIO  
from sklearn import linear_model,cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score

La descripción de los datos.

### Inputting the data 

In [9]:
data = pd.read_csv('./Data/datos_tc_uni.csv')

In [11]:
### Checking the structure of data

In [10]:
print('Estructura de los datos')
[(col, type(data[col][0])) for col in data.columns]

Estructura de los datos


[('LINEA_TC', numpy.int64),
 ('TIPO_FLUJO_TC', str),
 ('INGRESO_BRUTO', numpy.int64),
 ('EMP_REP_TC_SF', numpy.int64),
 ('EMP_REP_TC_BN', numpy.int64),
 ('EMP_REP_PP_SF', numpy.int64),
 ('EMP_REP_PP_BN', numpy.int64),
 ('EMP_REP_CONV_SF', numpy.int64),
 ('EMP_REP_CONV_BN', numpy.int64),
 ('EMP_REP_VEH_SF', numpy.int64),
 ('EMP_REP_VEH_BN', numpy.int64),
 ('EMP_REP_HIP_SF', numpy.int64),
 ('EMP_REP_HIP_BN', numpy.int64),
 ('FLG_TC_SF', numpy.int64),
 ('FLG_TC_BN', numpy.int64),
 ('FLG_PP_SF', numpy.int64),
 ('FLG_PP_BN', numpy.int64),
 ('FLG_CONV_SF', numpy.int64),
 ('FLG_CONV_BN', numpy.int64),
 ('FLG_VEH_SF', numpy.int64),
 ('FLG_VEH_BN', numpy.int64),
 ('FLG_HIP_SF', numpy.int64),
 ('FLG_HIP_BN', numpy.int64),
 ('FLG_BANCARIZADO', numpy.int64),
 ('FLG_CLI_SF', numpy.int64),
 ('FLG_CLI_BN', numpy.int64),
 ('CLF_SBS', numpy.float64),
 ('FLG_CLIENTE4', str),
 ('FLG_SEGURO', numpy.int64),
 ('CLI_AHOCRED', numpy.int64),
 ('CLI_CTS', numpy.int64),
 ('CLI_FM', numpy.int64),
 ('CLI_MILL', nu

In [ ]:
### First Stage-selection of market variables

In [20]:
data_market =data.drop(['LINEA_TC',
'TIPO_FLUJO_TC',
'EMP_REP_TC_BN',
'EMP_REP_PP_BN',
'EMP_REP_CONV_BN',
'EMP_REP_VEH_BN',
'EMP_REP_HIP_BN',
'FLG_TC_BN',
'FLG_PP_BN',
'FLG_CONV_BN',
'FLG_VEH_BN',
'FLG_HIP_BN',
'FLG_CLI_BN',
'FLG_CLIENTE4',
'FLG_SEGURO',
'CLI_AHOCRED',
'CLI_CTS',
'CLI_FM',
'CLI_MILL',
'CLI_PLA',
'CLI_PLAFI',
'CLI_TXS',
'NRO_PAS_BN',
'FLG_PAS',
'MON_MILL_U6',
'MON_PLA_U6',
'MON_FIJ_U6',
'MON_CTS_U6',
'MON_TRX_U6',
'MON_AHCRE_U6',
'MON_INV_U6',
'MON_MILL_M1',
'MON_PLA_M1',
'MON_FIJ_M1',
'MON_CTS_M1',
'MON_TRX_M1',
'MON_AHCRE_M1',
'MON_INV_M1',
'IND_EMP_REP_PP_BN',
'IND_EMP_REP_CONV_BN',
'IND_EMP_REP_VEH_BN',
'IND_EMP_REP_HIP_BN',
'IND_LINIBK_ING',
'PROB_CONT',
'CAMP_TOT',
'PROM_CAMP',
'RECENCIA_CAMP',
'NRO_CAMP_TC',
'CAMP_TC_U6',
'CREC_ENT_REP_TC',
'CREC_SAL_TC',
'SBS_CLF',
'CANALVTA_HOM',
'FLG_CAMP',
'CANAL_ASIGNADO',
'ARRIBO_PJE',
'SEXO',
'DEPARTAMENTO'], axis=1)

In [21]:
data_market.head()

,INGRESO_BRUTO,EMP_REP_TC_SF,EMP_REP_PP_SF,EMP_REP_CONV_SF,EMP_REP_VEH_SF,EMP_REP_HIP_SF,FLG_TC_SF,FLG_PP_SF,FLG_CONV_SF,FLG_VEH_SF,...,IND_EMP_REP_CONV_SF,IND_EMP_REP_VEH_SF,IND_EMP_REP_HIP_SF,SALDO_TOT_ACT,IND_MAX_SAL_TCSF_U6,IND_MAXLIN_ING,IND_DEU_ING,EDAD,SIT_LAB,TARGET_XF
0,2175,3,1,0,0,0,1,1,0,0,...,0,0,0,9608.25,1.0,7.816092,4.417586,26,DEPENDIENTE,1
1,2470,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0.0,0.000000,0.000000,44,INDEPENDIENTE,1
2,2145,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0.0,0.000000,0.000000,71,INDEPENDIENTE,1
3,1628,0,0,0,0,0,0,0,0,0,...,0,0,0,0.00,0.0,0.000000,0.000000,24,DEPENDIENTE,1
4,1500,3,0,0,0,0,1,0,0,0,...,0,0,0,639.82,1.0,0.466667,0.426547,27,INDEPENDIENTE,1


In [22]:
### Type of variables

In [23]:
print('Estructura de los datos')
[(col, type(data_market[col][0])) for col in data_market.columns]

Estructura de los datos


[('INGRESO_BRUTO', numpy.int64),
 ('EMP_REP_TC_SF', numpy.int64),
 ('EMP_REP_PP_SF', numpy.int64),
 ('EMP_REP_CONV_SF', numpy.int64),
 ('EMP_REP_VEH_SF', numpy.int64),
 ('EMP_REP_HIP_SF', numpy.int64),
 ('FLG_TC_SF', numpy.int64),
 ('FLG_PP_SF', numpy.int64),
 ('FLG_CONV_SF', numpy.int64),
 ('FLG_VEH_SF', numpy.int64),
 ('FLG_HIP_SF', numpy.int64),
 ('FLG_BANCARIZADO', numpy.int64),
 ('FLG_CLI_SF', numpy.int64),
 ('CLF_SBS', numpy.float64),
 ('SALDO_TC_SF', numpy.float64),
 ('SALDO_PP_SF', numpy.float64),
 ('SALDO_CONV_SF', numpy.float64),
 ('SALDO_VEH_SF', numpy.float64),
 ('SALDO_HIP_SF', numpy.float64),
 ('SALDO_TCSF_UU6', numpy.float64),
 ('MAX_SALDO_TCSF_U6', numpy.float64),
 ('MIN_SALDO_TCSF_U6', numpy.float64),
 ('MAX_LINEA_TCSF', numpy.float64),
 ('MIN_LINEA_TCSF', numpy.float64),
 ('LINEA_TOT_TCSF', numpy.float64),
 ('MAX_SALTC_SF_UM', numpy.float64),
 ('MIN_SALTC_SF_UM', numpy.float64),
 ('SALDO_SF_UM', numpy.float64),
 ('NRO_ENT_REP_TOT_U6', numpy.float64),
 ('NRO_ENT_REP_SA

In [7]:
## Create dummies for categorical variables

In [24]:
data_market.groupby('SIT_LAB').TARGET_XF.count()

SIT_LAB
0                  564
DEPENDIENTE      14586
INDEPENDIENTE    35889
MIXTO             9211
Name: TARGET_XF, dtype: int64

In [45]:
# create a DataFrame of dummy variables for "SITUACION LABORAL"
SL_DUMM = pd.get_dummies(data_market.SIT_LAB, prefix='SL_')
SL_DUMM.head()

,SL__0,SL__DEPENDIENTE,SL__INDEPENDIENTE,SL__MIXTO
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0


In [46]:
SL_DUMM.drop(SL_DUMM.columns[0],axis=1,inplace=True)
SL_DUMM.head()

,SL__DEPENDIENTE,SL__INDEPENDIENTE,SL__MIXTO
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [47]:
## Order the data for the analysis

In [48]:
mat_x=data_market.iloc[:,:-2]
mat_y=data_market.iloc[:,-1]

In [49]:
mat_x.head()

,INGRESO_BRUTO,EMP_REP_TC_SF,EMP_REP_PP_SF,EMP_REP_CONV_SF,EMP_REP_VEH_SF,EMP_REP_HIP_SF,FLG_TC_SF,FLG_PP_SF,FLG_CONV_SF,FLG_VEH_SF,...,NRO_ENT_REP_SAL_TOT_UM,IND_EMP_REP_PP_SF,IND_EMP_REP_CONV_SF,IND_EMP_REP_VEH_SF,IND_EMP_REP_HIP_SF,SALDO_TOT_ACT,IND_MAX_SAL_TCSF_U6,IND_MAXLIN_ING,IND_DEU_ING,EDAD
0,2175,3,1,0,0,0,1,1,0,0,...,3,1.0,0,0,0,9608.25,1.0,7.816092,4.417586,26
1,2470,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.00,0.0,0.000000,0.000000,44
2,2145,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.00,0.0,0.000000,0.000000,71
3,1628,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.00,0.0,0.000000,0.000000,24
4,1500,3,0,0,0,0,1,0,0,0,...,3,0.0,0,0,0,639.82,1.0,0.466667,0.426547,27


In [50]:
mat_y.head()

0    1
1    1
2    1
3    1
4    1
Name: TARGET_XF, dtype: int64

In [51]:
# concatenate the original DataFrame and the dummy DataFrame

In [52]:
data_market1 = pd.concat([mat_x,SL_DUMM], axis=1)
data_market1.head()

,INGRESO_BRUTO,EMP_REP_TC_SF,EMP_REP_PP_SF,EMP_REP_CONV_SF,EMP_REP_VEH_SF,EMP_REP_HIP_SF,FLG_TC_SF,FLG_PP_SF,FLG_CONV_SF,FLG_VEH_SF,...,IND_EMP_REP_VEH_SF,IND_EMP_REP_HIP_SF,SALDO_TOT_ACT,IND_MAX_SAL_TCSF_U6,IND_MAXLIN_ING,IND_DEU_ING,EDAD,SL__DEPENDIENTE,SL__INDEPENDIENTE,SL__MIXTO
0,2175,3,1,0,0,0,1,1,0,0,...,0,0,9608.25,1.0,7.816092,4.417586,26,1,0,0
1,2470,0,0,0,0,0,0,0,0,0,...,0,0,0.00,0.0,0.000000,0.000000,44,0,1,0
2,2145,0,0,0,0,0,0,0,0,0,...,0,0,0.00,0.0,0.000000,0.000000,71,0,1,0
3,1628,0,0,0,0,0,0,0,0,0,...,0,0,0.00,0.0,0.000000,0.000000,24,1,0,0
4,1500,3,0,0,0,0,1,0,0,0,...,0,0,639.82,1.0,0.466667,0.426547,27,0,1,0


In [53]:
# concatenate the original DataFrame and the dummy DataFrame
data_market2 = pd.concat([data_market1,mat_y], axis=1)
data_market2.head()

,INGRESO_BRUTO,EMP_REP_TC_SF,EMP_REP_PP_SF,EMP_REP_CONV_SF,EMP_REP_VEH_SF,EMP_REP_HIP_SF,FLG_TC_SF,FLG_PP_SF,FLG_CONV_SF,FLG_VEH_SF,...,IND_EMP_REP_HIP_SF,SALDO_TOT_ACT,IND_MAX_SAL_TCSF_U6,IND_MAXLIN_ING,IND_DEU_ING,EDAD,SL__DEPENDIENTE,SL__INDEPENDIENTE,SL__MIXTO,TARGET_XF
0,2175,3,1,0,0,0,1,1,0,0,...,0,9608.25,1.0,7.816092,4.417586,26,1,0,0,1
1,2470,0,0,0,0,0,0,0,0,0,...,0,0.00,0.0,0.000000,0.000000,44,0,1,0,1
2,2145,0,0,0,0,0,0,0,0,0,...,0,0.00,0.0,0.000000,0.000000,71,0,1,0,1
3,1628,0,0,0,0,0,0,0,0,0,...,0,0.00,0.0,0.000000,0.000000,24,1,0,0,1
4,1500,3,0,0,0,0,1,0,0,0,...,0,639.82,1.0,0.466667,0.426547,27,0,1,0,1


In [67]:
data_market3=data_market2.dropna(how='any')

In [68]:
data_model=data_market3.values
data_model[1:3]

array([[  2.47000000e+03,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.40000000e+01,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  2.14500000e+03,   0.00000000e+00,

In [69]:
## Separate the training and test dataset

In [70]:
X_train, X_test, y_train, y_test = train_test_split(data_model[:,:-1],data_model[:,-1], test_size = 0.3, random_state = 100)

In [71]:
## Create the Classic Predictive Model

In [72]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=3, min_samples_leaf=5)

clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')

In [ ]:
## Variable Importance

In [74]:
data_market3.columns.values

array(['INGRESO_BRUTO', 'EMP_REP_TC_SF', 'EMP_REP_PP_SF',
       'EMP_REP_CONV_SF', 'EMP_REP_VEH_SF', 'EMP_REP_HIP_SF', 'FLG_TC_SF',
       'FLG_PP_SF', 'FLG_CONV_SF', 'FLG_VEH_SF', 'FLG_HIP_SF',
       'FLG_BANCARIZADO', 'FLG_CLI_SF', 'CLF_SBS', 'SALDO_TC_SF',
       'SALDO_PP_SF', 'SALDO_CONV_SF', 'SALDO_VEH_SF', 'SALDO_HIP_SF',
       'SALDO_TCSF_UU6', 'MAX_SALDO_TCSF_U6', 'MIN_SALDO_TCSF_U6',
       'MAX_LINEA_TCSF', 'MIN_LINEA_TCSF', 'LINEA_TOT_TCSF',
       'MAX_SALTC_SF_UM', 'MIN_SALTC_SF_UM', 'SALDO_SF_UM',
       'NRO_ENT_REP_TOT_U6', 'NRO_ENT_REP_SAL_TOT_U6', 'PORC_ENT_REP_SAL',
       'NRO_ENT_REP_TOT_UM', 'NRO_ENT_REP_SAL_TOT_UM', 'IND_EMP_REP_PP_SF',
       'IND_EMP_REP_CONV_SF', 'IND_EMP_REP_VEH_SF', 'IND_EMP_REP_HIP_SF',
       'SALDO_TOT_ACT', 'IND_MAX_SAL_TCSF_U6', 'IND_MAXLIN_ING',
       'IND_DEU_ING', 'EDAD', 'SL__DEPENDIENTE', 'SL__INDEPENDIENTE',
       'SL__MIXTO', 'TARGET_XF'], dtype=object)

In [77]:
features = ['INGRESO_BRUTO', 'EMP_REP_TC_SF', 'EMP_REP_PP_SF',
       'EMP_REP_CONV_SF', 'EMP_REP_VEH_SF', 'EMP_REP_HIP_SF', 'FLG_TC_SF',
       'FLG_PP_SF', 'FLG_CONV_SF', 'FLG_VEH_SF', 'FLG_HIP_SF',
       'FLG_BANCARIZADO', 'FLG_CLI_SF', 'CLF_SBS', 'SALDO_TC_SF',
       'SALDO_PP_SF', 'SALDO_CONV_SF', 'SALDO_VEH_SF', 'SALDO_HIP_SF',
       'SALDO_TCSF_UU6', 'MAX_SALDO_TCSF_U6', 'MIN_SALDO_TCSF_U6',
       'MAX_LINEA_TCSF', 'MIN_LINEA_TCSF', 'LINEA_TOT_TCSF',
       'MAX_SALTC_SF_UM', 'MIN_SALTC_SF_UM', 'SALDO_SF_UM',
       'NRO_ENT_REP_TOT_U6', 'NRO_ENT_REP_SAL_TOT_U6', 'PORC_ENT_REP_SAL',
       'NRO_ENT_REP_TOT_UM', 'NRO_ENT_REP_SAL_TOT_UM', 'IND_EMP_REP_PP_SF',
       'IND_EMP_REP_CONV_SF', 'IND_EMP_REP_VEH_SF', 'IND_EMP_REP_HIP_SF',
       'SALDO_TOT_ACT', 'IND_MAX_SAL_TCSF_U6', 'IND_MAXLIN_ING',
       'IND_DEU_ING', 'EDAD', 'SL__DEPENDIENTE', 'SL__INDEPENDIENTE',
       'SL__MIXTO']

In [78]:
feat_importance=pd.DataFrame({'feature':features, 'importance':clf_gini.feature_importances_})

In [79]:
feat_importance.to_csv('features_importance.csv')

In [80]:
### Visualization Analytics

In [81]:
with open("tree_income.txt", "w") as f:
    f = export_graphviz(clf_gini, out_file=f,feature_names=features)

In [ ]:
### Performance of the Model

In [82]:
y_pred = clf_gini.predict(X_test)
y_pred

array([ 1.,  0.,  0., ...,  0.,  0.,  1.])

In [84]:
acc=accuracy_score(y_test,y_pred)*100
acc

70.493096646942803

In [86]:
gini=2*acc-100
gini

40.986193293885606